In [1]:
!pip install keras_vggface mtcnn gpt_2_simple

     |████████████████████████████████| 2.3MB 5.1MB/s 
     |████████████████████████████████| 675kB 49.3MB/s 
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7-cp36-none-any.whl size=23557 sha256=5d632809351365b5819771f8022f8a23855e5a0965cd9c01ed5b6e670e0f4c42
  Stored in directory: /root/.cache/pip/wheels/28/7f/89/1253cc7ae7fd1cdf130fa146ab17314fd2a5a6d48ccf21dec5
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609169 sha256=ae4ffaa40f1ff2d0728f93661aed80bdb8a6c00884c61640a3d1886dc03d3b54
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built gpt-2-simple regex


In [2]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
%cd "/content/drive/My Drive/SI671 Final Project/Project"

/content/drive/My Drive/SI671 Final Project/Project


In [10]:
%tensorflow_version 1.x
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
from mtcnn.mtcnn import MTCNN

from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array

from keras_vggface.vggface import VGGFace
import keras_vggface.utils as vggface_utils

import spacy

import json
import os
import tensorflow as tf
import random
import model, sample, encoder

Using TensorFlow backend.


In [0]:
def get_gpt_response(
    model_name='345M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    top_p=1,
    models_dir='models',
    raw_text=""
):
    """
    Interactively run the model
    :model_name=124M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)
    """
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k, top_p=top_p
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        
        context_tokens = enc.encode(raw_text)
        generated = 0
        for _ in range(nsamples // batch_size):
            out = sess.run(output, feed_dict={
                context: [context_tokens for _ in range(batch_size)]
            })[:, len(context_tokens):]
            for i in range(batch_size):
                generated += 1
                text = enc.decode(out[i])
        return text

In [0]:
def get_objects(model, filename, size):
    image = load_img(filename, target_size=size)
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    yhat = model.predict(image)
    label = decode_predictions(yhat)

    return label

In [0]:
def detect_faces(model, filename, size):
    pixels = plt.imread(filename)
    results = model.detect_faces(pixels)

    x1, y1, width, height = results[0]["box"]
    x2, y2 = x1 + width, y1 + height

    face = pixels[y1:y2, x1:x2]

    image = Image.fromarray(face)
    image = image.resize(size)
    face_array = np.asarray(image)

    return face_array

In [0]:
def recognize_faces(model, face_array):
    pixels = face_array.astype("float32")
    
    samples = np.expand_dims(pixels, axis=0)
    samples = vggface_utils.preprocess_input(samples, version=2)
    
    yhat = model.predict(samples)
    
    results = vggface_utils.decode_predictions(yhat)

    return results

In [0]:
def get_list_of_objects(raw_objects):
    objects = []
    for obj in raw_objects[0]:
        labels = obj[1].split("_")
        objects.extend(labels)
    return objects

In [0]:
def get_stories(database, objects):
    search_condition = "|".join(objects)
    results = database[database["sentence1"].str.contains(search_condition)]
    seed = results.sample()["sentence1"].iloc[0]
    return seed

In [0]:
get_stories(database, ['dress'])

'Robin needed a new dress for the Civil War themed dance.'

In [0]:
def get_face(faces):
    face = faces[0][0][0].split("b'")[1][1:-1].split("_")[0]
    return face

In [0]:
def get_entities(seed):
    doc = nlp(seed)
    named_entities = [ent for ent in doc.ents if ent.label_ in ["PERSON", "ORG"]]
    pronouns = [token for token in doc if token.pos_ == "PRON"]
    return named_entities, pronouns

In [0]:
def get_modified_seed(seed, person, named_entities, pronouns):
    if len(named_entities) > 0:
        to_replace = named_entities[0].text
    else:
        if len(pronouns) > 0:
            to_replace = pronouns[0].text
        else:
            to_replace = ">>>>"
    modified_seed = seed.replace(to_replace, person)
    return modified_seed

In [0]:
def generate_story(seed):
    story = get_gpt_response(model_name="run1", 
                             models_dir="weights/ss/", raw_text=seed)
    story = story.split("\n")[0]
    return seed + story

In [0]:
object_model = VGG16()
face_detector_model = MTCNN()
face_model = VGGFace(model='resnet50')
database = pd.read_csv("data/combined.csv")
nlp = spacy.load("en")






553467904/553467096 [==============================] - 26s 0us/step









165445632/165439116 [==============================] - 3s 0us/step


In [0]:
def create_story_from_image(filename, size=(224, 224)):
    # Get the objects
    raw_objects = get_objects(object_model, filename, size)
    objects = get_list_of_objects(raw_objects)[:3]

    # Identify faces if any
    face_array = detect_faces(face_detector_model, filename, size)
    faces = recognize_faces(face_model, face_array)

    # Get proper name of identified face
    person = get_face(faces)

    # Retrieve relevant stories from database
    #seed = get_stories(database, objects)
    seed = get_stories(database, objects)

    # Get named entities and pronouns from seed
    named_entities, pronouns = get_entities(seed)

    # Identify named entities in retrieved story
    modified_seed = get_modified_seed(seed, person, named_entities, pronouns)

    # Debug and check
    print("Faces", faces)
    print("Person", person)
    print("Objects", objects)
    print("Seed", seed)
    print("Named Entities", named_entities)
    print("Pronouns", pronouns)
    print("Modified Seed", modified_seed)

    # Finally, generate Story from GPT-2
    story = generate_story(modified_seed)

    return story

In [0]:
# create_story_from_image("data/brad.jpg")

In [0]:
# create_story_from_image("data/sharon.jpg")

Faces [[["b' Sharon_Stone'", 0.9964234], ["b' Noelle_Reno'", 0.00084738154], ["b' Elisabeth_R\\xc3\\xb6hm'", 0.00032624512], ["b' Anita_Lipnicka'", 0.0002560317], ["b' Tina_Maze'", 0.000194267]]]
Person Sharon
Objects ['miniskirt', 'cardigan', 'gown', 'velvet', 'poncho']
Seed Joey was getting married and needed a gown.
Named Entities [Joey]
Pronouns []
Modified Seed Sharon was getting married and needed a gown.
INFO:tensorflow:Restoring parameters from weights/ss/run1/model-1000


'Sharon was getting married and needed a gown. She asked her friend for help. Her friend put some glitter around her neck and brought a dress to sell. Sharon was pleased with the dress.'

In [0]:
# create_story_from_image("data/tatum.jpg")

Faces [[["b' Channing_Tatum'", 0.9443253], ["b' Eoghan_Quigg'", 0.0014580755], ["b' Les_Miles'", 0.0011346485], ["b' Ibrahim_Afellay'", 0.0007187865], ["b' Tovah_Feldshuh'", 0.0006971031]]]
Person Channing
Objects ['chain', 'mail', 'crutch', 'reel', 'microphone', 'gar']
Seed Ulrich was trying to adjust his microphone stand.
Named Entities [Ulrich]
Pronouns []
Modified Seed Channing was trying to adjust his microphone stand.
INFO:tensorflow:Restoring parameters from weights/ss/run1/model-1000


"Channing was trying to adjust his microphone stand. One foot went sliding by him's ears. He lost control and wasn't able to get the balance right before hit."

In [0]:
# create_story_from_image("data/katy.jpg")